# Initial Imports & Setup

In [33]:
import pandas as pd
import numpy as np
import glob as glob
import matplotlib.pyplot as plt
import matplotlib.font_manager as font_manager
import seaborn as sns
import datetime as datetime

# %matplotlib inline
plt.style.use('fivethirtyeight')
plt.rcParams["svg.fonttype"] = "none"
plt.rcParams["font.family"] = "Bitstream Vera Sans"
plt.rcParams["figure.titleweight"] = 'bold'
plt.rcParams['figure.titlesize'] = 'xx-large'

# Importing Data into Dataframe

Still working on whether to index or not. Maybe all columns should be data for now. 

In [38]:
files = glob.glob('./data/*.csv')

df = pd.DataFrame()
for file in files:
    filedf = pd.read_csv(file,
                        usecols=["NAME","Metricname","SeenNum","SeenDenom"])
    # Prettier Names
    filedf = filedf.rename(columns={'NAME': 'MeridiosName',
                                    'Metricname': 'Metric',
                                    'SeenNum': 'Numerator', 
                                    'SeenDenom': 'Denominator'})
    filename_parts = file[7:-4].split(' ')
    # 0 Padded Dates with Dots. 03.15.2018
    if (len(str(filename_parts[0])) is 10):
        filedf['Date'] = datetime.datetime.strptime(filename_parts[0], "%m.%d.%Y")
        filedf['Percentage'] = filedf['Numerator'] / filedf['Denominator'] * 100
        #Not sure how to handle data vs index
        #filedf.set_index(['MeridiosName', 'Metric', 'Date'], inplace=True)
        df = df.append(filedf) 
    else:
        print("CSV Data File doesn't have Zero-Padded Date at beginning.")
#df.dtypes
#df.head()
#df.info()
#df.describe()
#df.columns
#df.Metric.unique()

# providergender = pd.read_csv('./current/provider-gender.csv', index_col='NAME')
# currentdata['Provider-gender']=currentdata.NAME.map(providergender.GENDER)


# Understanding the data

Indexes need to be unique so a heirarchical index of Name, Metric and Date would be unique. 

Will need to use GroupBy function of Pandas

### Lookups:

* what clinic does a provider belong to? 
* Person vs Clinic
* What's the Provider's 'Real Name' 
* What's a short name for the Metric?
* What's a target percentage for a measure

### Cleanups:

* Percentage can be rounded to 2 decimal points (but why throw out the data on the import?)


### Validations:

* Does Data include new MeridiosName, New Metric items? (Has the underlying report changed?)



In [27]:
Provider = df[df['MeridiosName'] == 'Ploudre MD, Jonathan']
for number, metric in enumerate(Provider.Metric.unique()):
    ProviderMetric = Provider[Provider['Metric'] == metric]
    ax = ProviderMetric.plot(x="Date", y="Percentage", title=metric, legend=False, ylim=(0,100), figsize=(6,6))
    ax.set_xlabel("")
    
    # Let's Do ever third tick mark (quarters)
    ax.set_xticks(ax.get_xticks()[::3])
    fig = ax.get_figure()
    fig.patch.set_facecolor('none')
    ax.patch.set_facecolor('white')
    ax.patch.set_alpha(0.8)
    fig.tight_layout()
    fig.savefig(str(number) + ".svg", facecolor=fig.get_facecolor(), edgecolor='none')
    plt.close(fig)
    